In [1]:
# DEFINE FUNCTIONS <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

def get_access_token(client_id:str, client_sc:str):
    import requests
    
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
    }
    data = f'grant_type=client_credentials&client_id={client_id}&client_secret={client_sc}'.encode()
    response = requests.post('https://accounts.spotify.com/api/token', headers=headers, data=data).json()
    access_token = response['access_token']

    return access_token

def get_response(access_token:str, endpoint:str, params:dict=None):
    import requests, json

    url = f"https://api.spotify.com/v1/{endpoint}"
    headers = {
        'Authorization': f'Bearer {access_token}',
    }

    if params != None:
        response = requests.get(url=url, params=params, headers=headers)
    else:
        response = requests.get(url=url, headers=headers)
    print(response)
    return response.json()
    

def post_response(access_token:str, endpoint:str, data:dict=None):
    import requests, json

    url = f"https://api.spotify.com/v1/{endpoint}"
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json"
    }

    response = requests.post(url=url, headers=headers, data=json.dumps(data))
    print(response)
    return response.json()

In [2]:
import os
from configparser import ConfigParser
current_dir = os.getcwd()
config_dir = os.path.join(current_dir, f'../config/config.ini')

parser = ConfigParser()
parser.read(config_dir)
client_id = parser.get("SPOTIFY", "ACCESS")
client_sc = parser.get("SPOTIFY", "SECRET")

access_token = get_access_token(client_id, client_sc)
user_id = "317sdvs4ldhgrv63cstoyzf6vo2y"

In [3]:
# START CODE <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from math import ceil
import json

In [4]:
### Build Session
spark = SparkSession.builder \
    .appName("pipeline_demo") \
    .getOrCreate()

### Create Access Token
access_token = get_access_token(client_id=client_id, client_sc=client_sc)

### Create Playlist Lists
endpoint = f"users/{user_id}/playlists"
params = {
    "limit": 50,
    "offset": 0
}

playlists = get_response(access_token=access_token, endpoint=endpoint, params=params)
json_string  = json.dumps(playlists)
json_rdd = spark.sparkContext.parallelize([json_string])
df_plinfo = spark.read.json(json_rdd, multiLine=True)

items = df_plinfo \
    .withColumn("items", explode("items")) \
    .select("items.id") \
    .rdd.flatMap(lambda x: x).collect()

### Create Playlist Item Lists
track_list = [] # <---------- "Need To Use"
for id in items:
    endpoint = f"playlists/{id}/tracks"
    playlist_spec = get_response(access_token=access_token, endpoint=endpoint)
    
    json_string  = json.dumps(playlist_spec)
    json_rdd = spark.sparkContext.parallelize([json_string])
    df_playlist_spec = spark.read.json(json_rdd, multiLine=True)
    
    ids = df_playlist_spec \
    .withColumn("items", explode("items")) \
    .select("items.track.id") \
    .rdd.flatMap(lambda x: x).collect()
    
    track_list += ids
    
    total = df_playlist_spec.select("total").first()[0]
    print(total)
    left = int(total)-100
    cnt = ceil(left/100)
    
    for i in range(cnt):
        offset = 100 + 100 * i
        params = {"offset":offset}
        
        playlist_spec = get_response(access_token=access_token, endpoint=endpoint, params=params)
        
        json_string  = json.dumps(playlist_spec)
        json_rdd = spark.sparkContext.parallelize([json_string])
        df_playlist_spec = spark.read.json(json_rdd, multiLine=True)
        
        ids = df_playlist_spec \
        .withColumn("items", explode("items")) \
        .select("items.track.id") \
        .rdd.flatMap(lambda x: x).collect()
        
        track_list += ids      


131072x1 화면 크기가 잘못됐습니다. 문제가 예상됩니다
24/01/12 02:09:25 WARN Utils: Your hostname, KJH-DESKTOP resolves to a loopback address: 127.0.1.1; using 192.168.69.220 instead (on interface eth0)
24/01/12 02:09:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/12 02:09:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/01/12 02:09:27 WARN RapidsPluginUtils: RAPIDS Accelerator 23.12.0 using cudf 23.12.0.
24/01/12 02:09:27 WARN RapidsPluginUtils: spark.rapids.sql.multiThreadedRead.numThreads is set to 20.
24/01/12 02:09:27 WARN RapidsPluginUtils: RAPIDS Accelerator is enabled, to disable GPU support set `spark.rapids.sql.enabled` to false.


<Response [200]>


<Response [200]>
41
<Response [200]>
122
<Response [200]>
<Response [200]>
191
<Response [200]>


In [5]:
track_list = [x for x in track_list if x is not None]
track_set = set(track_list)
track_list = list(track_set)
len(track_list)

211

In [6]:
# Create Dataframe : main_df
big_list = []
cnt = ceil(len(track_list)/50)
for j in range(cnt):
    big_list.append(track_list[j*50:(j+1)*50])


main_df = None
cnt = 0
for small_list in big_list:
    
    print(cnt)
    
    tracks = ""
    for id in small_list:
        tracks += f",{id}"
    tracks = tracks[1:]
    
    endpoint = "tracks"
    params = {"ids":tracks}
    track = get_response(access_token=access_token, endpoint=endpoint, params=params)
    
    json_string  = json.dumps(track)
    json_rdd = spark.sparkContext.parallelize([json_string])
    df_tracks = spark.read.json(json_rdd, multiLine=True)
    
    df_tracks = spark.read.json(json_rdd, multiLine=True) \
        .withColumn("tracks", explode("tracks")) \
        .selectExpr("tracks.id",
                    "tracks.popularity")
    
    endpoint = "audio-features"
    params = {"ids":tracks}
    audio_features = get_response(access_token=access_token, endpoint=endpoint, params=params)
    
    json_string  = json.dumps(audio_features)
    json_rdd = spark.sparkContext.parallelize([json_string])
    df_audio_features = spark.read.json(json_rdd, multiLine=True) \
        .withColumn("audio_features", explode("audio_features")) \
        .selectExpr("audio_features.id",
                    "audio_features.key",
                    "audio_features.mode",
                    "audio_features.time_signature",
                    "audio_features.tempo",
                    "audio_features.acousticness",
                    "audio_features.danceability",
                    "audio_features.energy",
                    "audio_features.instrumentalness",
                    "audio_features.liveness",
                    "audio_features.loudness",
                    "audio_features.speechiness",
                    "audio_features.valence")
    
    result_track_df = df_tracks.join(df_audio_features, "id", "left")
    if cnt == 0:
        main_df = result_track_df
    else:
        main_df = main_df.union(result_track_df)
    cnt += 1

0
<Response [200]>
<Response [200]>
1
<Response [200]>
<Response [200]>
2
<Response [200]>
<Response [200]>
3
<Response [200]>
<Response [200]>
4
<Response [200]>
<Response [200]>


In [7]:
main_df.show()

+--------------------+----------+---+----+--------------+-------+------------+------------+------+----------------+--------+--------+-----------+-------+
|                  id|popularity|key|mode|time_signature|  tempo|acousticness|danceability|energy|instrumentalness|liveness|loudness|speechiness|valence|
+--------------------+----------+---+----+--------------+-------+------------+------------+------+----------------+--------+--------+-----------+-------+
|0jydFAxRpAZzSdHdN...|        13|  9|   1|             4|118.997|       0.142|       0.839|  0.47|            0.77|   0.126|  -9.427|      0.048|  0.503|
|6RBziRcDeiho3iTPd...|        61| 11|   0|             4| 97.977|      0.0758|       0.534| 0.861|             0.0|   0.167|  -3.177|     0.0361|  0.286|
|013AWvizllIUEC2FO...|        72|  0|   1|             4| 168.12|       0.403|       0.555| 0.698|             0.0|  0.0851|  -8.874|     0.0287|  0.816|
|0BQ3kSfeLFq3CMtuH...|        39|  2|   1|             3| 70.018|       0.99

In [8]:
### Load Dataframe : df_dw
dw_tracks = spark.read.parquet("file:///home/kjh/data/Spotify/tracks")
dw_audioFeatures = spark.read.parquet("file:///home/kjh/data/Spotify/tracks_audioFeatures")
df_dw = dw_tracks.join(dw_audioFeatures, "id", "inner")

df_dw.show() # << TEST

+--------------------+----------+---+----+--------------+-------+------------+------------+-------+----------------+--------+--------+-----------+-------+
|                  id|popularity|key|mode|time_signature|  tempo|acousticness|danceability| energy|instrumentalness|liveness|loudness|speechiness|valence|
+--------------------+----------+---+----+--------------+-------+------------+------------+-------+----------------+--------+--------+-----------+-------+
|3au7XGR06xFLlHQ1e...|         0|  1|   0|             4| 96.514|       0.879|       0.282|  0.251|          0.0495|  0.0674| -21.701|     0.0313|  0.665|
|36pujsnGRtaHUecVb...|         0|  2|   1|             4|114.984|     0.00708|       0.878|  0.883|           0.141|   0.112|  -4.934|     0.0755|  0.423|
|740kpbO9VNamJ3I63...|         0|  5|   1|             4|101.381|       0.819|       0.726|  0.809|          0.0783|   0.332|  -4.921|     0.0462|  0.733|
|014iV3BRCZeMGMOLY...|         0|  0|   1|             4|112.018|     

In [9]:
df_dw = df_dw.dropDuplicates(['id'])

In [10]:
df_dw.createOrReplaceTempView("df_dw")

# track_list를 SQL 쿼리에 맞는 형식으로 변환
track_list_str = ",".join(["'{}'".format(item) for item in track_list])

In [11]:
df = spark.sql(f"""
    SELECT *
    FROM df_dw
    WHERE id NOT IN ({track_list_str})
""")

In [16]:
### Union Dataframe : df
from pyspark.sql.functions import col

df = df_dw \
    .filter(~df_dw["id"].isin(track_list)) \
    .union(main_df)

df = df_dw \
    .filter(~df_dw["id"].isin(track_list)) \
    .union(main_df)

위 코드가 작동이 안되었던 이유는 track_list의 값에 None 값이 있기 때문..

https://stackoverflow.com/questions/74272965/using-not-is-in-in-pyspark-and-getting-an-empty-dataframe-back

In [18]:
# df = df_dw \
    # .filter(~df_dw["id"].isin(track_list)) \
    # .union(main_df)
    
# 위 코드가 작동이 안되었던 이유는 track_list

In [19]:
df.count()

1581533

In [20]:
# 고려해야 할 사항들
"""
    0. 스케일링 가능 항목 : acousticness, danceability, energy, instrumentalness, liveness, speechiness, valence
    1. loudness 항목 : 절댓값 반환 필요 / 계산식 확인 필요
            from pyspark.sql.functions import abs
            df = df \
                .withColumn("loudness", abs("loudness"))
    2. mode 항목 : 0 또는 1만 가짐
    3. key, tempo 항목 : 이상치가 큰 영향을 끼칠 수 있음
    4. Null 값 확인
            #1번 - scaled_df.filter(col("features").isNull()).count()
            #2번 - from pyspark.sql.functions import sum
                   df \
                       .select([sum(col(column).isNull().cast("int")).alias(column + "_null_count") for column in df.columns]) \
                       .show()
"""

'\n    0. 스케일링 가능 항목 : acousticness, danceability, energy, instrumentalness, liveness, speechiness, valence\n    1. loudness 항목 : 절댓값 반환 필요 / 계산식 확인 필요\n            from pyspark.sql.functions import abs\n            df = df                 .withColumn("loudness", abs("loudness"))\n    2. mode 항목 : 0 또는 1만 가짐\n    3. key, tempo 항목 : 이상치가 큰 영향을 끼칠 수 있음\n    4. Null 값 확인\n            #1번 - scaled_df.filter(col("features").isNull()).count()\n            #2번 - from pyspark.sql.functions import sum\n                   df                        .select([sum(col(column).isNull().cast("int")).alias(column + "_null_count") for column in df.columns])                        .show()\n'

In [21]:
from pyspark.sql.functions import mean, stddev

raw_scaled_df = df

# Standard Scaling
for col_name in df.columns[1:]:
    mean_val = df.select(mean(col_name)).collect()[0][0]
    stds_val = df.select(stddev(col_name)).collect()[0][0]
    raw_scaled_df = raw_scaled_df.withColumn(col_name, (col(col_name) - mean_val) / stds_val)

raw_scaled_df.show()

24/01/12 02:28:24 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+------------+------------+-----------+--------------+------------+------------+------------+------------+----------------+------------+------------+------------+------------+
|                  id|  popularity|         key|       mode|time_signature|       tempo|acousticness|danceability|      energy|instrumentalness|    liveness|    loudness| speechiness|     valence|
+--------------------+------------+------------+-----------+--------------+------------+------------+------------+------------+----------------+------------+------------+------------+------------+
|0007EK8kcv06GzJdS...|-0.338179953|-1.473045612|0.751636992|  -1.645467302| 1.172293687| 1.420362129|-0.702576667| 1.061979849|    -0.398533921| 1.006833389|-0.887245328| 4.932819349|-1.522515973|
|0007aPK8VmXN4ycL2...| 0.773623017|-1.473045612|0.751636992|   0.261698998|-0.074274998|-1.119285791|-0.318041294| 1.571018331|    -0.165066441| 4.427973268| 0.751401424|-0.107435312|-0.336431148|
|0007zfsbbIVYtz

In [22]:
from pyspark.sql.functions import col

scaled_train = raw_scaled_df.filter(col("id").isin(track_list))
scaled_test = raw_scaled_df.filter(~col("id").isin(track_list))

In [23]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans

selected_features = ["popularity", "key", "mode", "time_signature", "tempo", "acousticness", "danceability", "energy", "instrumentalness", "liveness", "loudness", "speechiness", "valence"]

# Define Assembler
assembler = VectorAssembler(inputCols=selected_features, outputCol="features")

# Assemble Features
df_assembled_train = assembler.transform(scaled_train)
df_assembled_test = assembler.transform(scaled_test)

# Create Model == Train Dataset
kmeans = KMeans(featuresCol="features", k=4, seed=1)
model = kmeans.fit(df_assembled_train)

# Check Centers
centers = model.clusterCenters()
print("Cluster Centers:")
for center in centers:
    print(center)

# Test Dataset
df_result = model.transform(df_assembled_test)
df_result.show()

24/01/12 02:28:39 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


Cluster Centers:
[ 2.87487021  0.95883695  0.02744026  0.22023886  0.07254316 -0.83176664
 -0.12338519  0.95942936 -0.42570345  0.03964644  0.79146922 -0.38263627
  0.14678447]
[ 0.79267609 -0.21275569  0.18074803  0.2001775   0.05805026 -0.3690155
  0.28751734  0.49721256 -0.52355248 -0.14404919  0.81678342 -0.43179473
  0.37584837]
[ 4.2183896  -0.53767534  0.10312475  0.261699    0.15500959 -0.80775275
  0.56900166  0.91097158 -0.63112137 -0.17488138  0.90084074 -0.24030983
  0.69035448]
[ 2.95917618  0.20000114  0.55334503  0.03465539  0.0047358   0.15036533
 -0.44229592 -0.43101707 -0.13202886 -0.38405979  0.20192772 -0.51935477
 -0.62566121]


+--------------------+--------------------+--------------------+------------------+-------------------+--------------------+-------------------+--------------------+-------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+----------+
|                  id|          popularity|                 key|              mode|     time_signature|               tempo|       acousticness|        danceability|             energy|    instrumentalness|            liveness|           loudness|         speechiness|             valence|            features|prediction|
+--------------------+--------------------+--------------------+------------------+-------------------+--------------------+-------------------+--------------------+-------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+----------+
|0007EK8kcv06GzJdS...|   -0.338179

In [24]:
model.getDistanceMeasure()

'euclidean'

In [25]:
from pyspark.sql.functions import col
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType
import numpy as np

def calculate_distance(vector):
    return float(np.linalg.norm(vector.toArray() - numpy_coordinates))

calculate_distance_udf = udf(calculate_distance, DoubleType())

recommend_list = []
for i in range(4):
    numpy_coordinates = centers[i]
    df_with_distance = df_result \
        .filter(f"prediction={i}") \
        .select("id", "features") \
        .withColumn('distance', calculate_distance_udf(col('features'))) \
        .orderBy("distance", ascending=True)
    
    collect_list = [row.id for row in df_with_distance.limit(5).collect()]

    print(collect_list)
    recommend_list += collect_list
    

['5BAIGUYfFlUeuo16kFuan0', '26MwBGP2wbNUDZhSS43wAn', '6CLi7ysa6YNvoBlYg9W5Qc', '3N260ZBbFQOypRyKSQaIGB', '0VNb0HzD2oxJ9zyRbv7xFH']


['3J76VaaKdpIlhschNLbw3E', '10WBJf270bCgyDI1EuGP28', '22nCqMhCqGi3YkMMdXNRBx', '4vdEK97UUC3N4tabBuYcVs', '35fNhpzCPJIV3LWZzWUQXI']


['0oPRfdP48RM4bd3xBFyiRo', '5goqVeMYUQecvpcjNxpadU', '5D7PM5yNiHKnU85Y055vvb', '0is4ka3uSbBT04pC2WFsp3', '1pRShUtGOG6ZP5fsnw9G4h']


['4PqGEhQlHBczN9hBbui8eY', '37OwL4VPwKjyqu0VaqEizD', '0GfCR4T6WOErZOw1557ln7', '05O75TNqW8S6sYYvoCDDIe', '4a1RWaG4BTkifgMSx3rpf3']


In [26]:
print(recommend_list)

['5BAIGUYfFlUeuo16kFuan0', '26MwBGP2wbNUDZhSS43wAn', '6CLi7ysa6YNvoBlYg9W5Qc', '3N260ZBbFQOypRyKSQaIGB', '0VNb0HzD2oxJ9zyRbv7xFH', '3J76VaaKdpIlhschNLbw3E', '10WBJf270bCgyDI1EuGP28', '22nCqMhCqGi3YkMMdXNRBx', '4vdEK97UUC3N4tabBuYcVs', '35fNhpzCPJIV3LWZzWUQXI', '0oPRfdP48RM4bd3xBFyiRo', '5goqVeMYUQecvpcjNxpadU', '5D7PM5yNiHKnU85Y055vvb', '0is4ka3uSbBT04pC2WFsp3', '1pRShUtGOG6ZP5fsnw9G4h', '4PqGEhQlHBczN9hBbui8eY', '37OwL4VPwKjyqu0VaqEizD', '0GfCR4T6WOErZOw1557ln7', '05O75TNqW8S6sYYvoCDDIe', '4a1RWaG4BTkifgMSx3rpf3']


In [27]:
import requests
from urllib.parse import urlencode
import base64
import webbrowser

client_id = "a18dc6eb0d9042eaa547bad38ee1b237"

In [28]:
def create_auth_code(client_id:str, scope:str):
    from urllib.parse import urlencode
    import webbrowser

    auth_headers = {
        "client_id": client_id,
        "response_type": "code",
        "redirect_uri": "http://localhost:1111",
        "scope": scope
    }

    webbrowser.open("https://accounts.spotify.com/authorize?" + urlencode(auth_headers))
    
create_auth_code(client_id=client_id, scope="playlist-modify-private")

In [29]:
code = "AQDNRcy-nkg1TkoARhtKjUMoALfauUnY1R3o4qb-Q42fDvmPP2PwOOg217GvNXSXFe5RMxJ1vhkYRPdl1LbELN7T_7NlDT3GuwTM6-tCv-5-I6Q_3Z5gLBBQkmbhozX4zr1C5PRYDX8MW1dI2sq3uzAmicJXlIu6xU6CFbDRWNESLceNSO5cyjhjtZ1bes-8HeM"

encoded_credentials = base64.b64encode(client_id.encode() + b':' + client_sc.encode()).decode("utf-8")

token_headers = {
    "Authorization": "Basic " + encoded_credentials,
    "Content-Type": "application/x-www-form-urlencoded"
}

token_data = {
    "grant_type": "authorization_code",
    "code": code,
    "redirect_uri": "http://localhost:1111"
}

r = requests.post("https://accounts.spotify.com/api/token", data=token_data, headers=token_headers)


response_json = r.json()
print(response_json)

{'access_token': 'BQA1R6lDBct6Uit0o9zVqSJpDJuxJRCb-2XUQWc5SGSkWuJmM8h1IAQxWscPx3ZEbyjiuwHLUjY2VPgjsy3pTC_6_-vDiPYUln5rF3wVJPWqsqPgIlKVpAkMWoNgUJeolKTAEr71estIpB2PDkDNiQhD20BqvOzcFjSsgU7MJYyhNdG4c1fH2duABWE_Krz2wZEy5eVzHYNHuveV37Ko852_EJ20EEIJVBhXXRlutw', 'token_type': 'Bearer', 'expires_in': 3600, 'refresh_token': 'AQDq9qAdaiRGMHkLXwCcKfgyzdyeBDZodTX6C6rr3GEQ1dV7QvorBCU64AlPxWNJ7XtFxIhU4_GUSe9l1nIdE_lfDzK-0lD8NPV4no2u9VmfdAGR9VKluJeLIRX_3Qp48ks', 'scope': 'playlist-modify-private'}


In [30]:
token = r.json()["access_token"]

In [31]:
from datetime import datetime

today = datetime.now()
first_day_of_month = today.replace(day=1)
day_difference = (today - first_day_of_month).days
week_number = (day_difference // 7) + 1
year = today.strftime("%Y")
month = today.strftime("%m")

endpoint = f"users/{user_id}/playlists"
playlist_name = f"{year}년 {month}월 {week_number}주차 추천 플레이리스트"
playlist_description = "고객님의 플레이리스트 기록을 기반으로 매주 음악을 추천해 드립니다 :)"
data = {
    "name": playlist_name,
    "description": playlist_description,
    "public": False
}

returned = post_response(access_token=token, endpoint=endpoint, data=data)
print(returned)

<Response [201]>
{'collaborative': False, 'description': '고객님의 플레이리스트 기록을 기반으로 매주 음악을 추천해 드립니다 :)', 'external_urls': {'spotify': 'https://open.spotify.com/playlist/434eVpu05FRJbWaoqym1Oq'}, 'followers': {'href': None, 'total': 0}, 'href': 'https://api.spotify.com/v1/playlists/434eVpu05FRJbWaoqym1Oq', 'id': '434eVpu05FRJbWaoqym1Oq', 'images': [], 'name': '2024년 01월 2주차 추천 플레이리스트', 'owner': {'display_name': '주헌', 'external_urls': {'spotify': 'https://open.spotify.com/user/317sdvs4ldhgrv63cstoyzf6vo2y'}, 'href': 'https://api.spotify.com/v1/users/317sdvs4ldhgrv63cstoyzf6vo2y', 'id': '317sdvs4ldhgrv63cstoyzf6vo2y', 'type': 'user', 'uri': 'spotify:user:317sdvs4ldhgrv63cstoyzf6vo2y'}, 'primary_color': None, 'public': False, 'snapshot_id': 'MSxiOTNlYWI2MjVjNWQwYWJhZWUxZTZiMzQwZTkxNjhjNDY3OWEwZjll', 'tracks': {'href': 'https://api.spotify.com/v1/playlists/434eVpu05FRJbWaoqym1Oq/tracks', 'items': [], 'limit': 100, 'next': None, 'offset': 0, 'previous': None, 'total': 0}, 'type': 'playlist', 'uri

In [32]:
playlist_id = returned["id"]

In [33]:
endpoint = f"playlists/{playlist_id}/tracks"
data = {
    "uris": [f"spotify:track:{track_id}" for track_id in recommend_list],
    "position": 0
}

post_response(access_token=token, endpoint=endpoint, data=data)

<Response [201]>


{'snapshot_id': 'MyxlMjY4NjcyZDI4YWNlNDg2MTBkNmU5YjljNDZmMWNiZjA2ZDdhYWIy'}

In [ ]:
spark.stop()